#### Import the necessary libraries

In [285]:
#set the parent folder
import sys
sys.path.append('..')

In [286]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from util.methods import build_frequency_table ,build_cdf
from util.classes import UniformQuantizerByRange, StaticANSEncoder, StaticANSDecoder
from util.evaluator import Evaluator

#### Load the Dataset

In [287]:
AriviyalN_Data_Wind_Direction = pd.read_csv(r"..\datasets\AriviyalN_Data_Wind_Direction.csv",header=0)
AriviyalN_Data_Wind_Direction.head(10)

,Timestamp,Wind Direction
0,28/02/2024 11:15,81.0
1,28/02/2024 11:30,71.0
2,28/02/2024 11:45,68.0
3,28/02/2024 12:00,64.0
4,28/02/2024 12:15,79.0
5,28/02/2024 12:30,61.0
6,28/02/2024 12:45,72.0
7,28/02/2024 13:00,61.0
8,28/02/2024 13:15,67.0
9,28/02/2024 13:30,68.0


### Split the dataset

In [288]:
train_AriviyalN_Data_Wind_Direction , test_AriviyalN_Data_Wind_Direction = train_test_split(AriviyalN_Data_Wind_Direction,test_size=0.2,random_state=42)
print(len(train_AriviyalN_Data_Wind_Direction))
print(len(test_AriviyalN_Data_Wind_Direction))

41083
10271


### Quantize the data

In [289]:
min_value = np.min(train_AriviyalN_Data_Wind_Direction["Wind Direction"])
max_value = np.max(train_AriviyalN_Data_Wind_Direction["Wind Direction"])
q = UniformQuantizerByRange(min_value, max_value)


#### Create a frequency table

In [290]:
data_list = train_AriviyalN_Data_Wind_Direction["Wind Direction"].to_numpy().tolist()
value_to_symbol_list = [q.value_to_symbol(i) for i in data_list]
freq_table = build_frequency_table(data_list,q)
total,cdf = build_cdf(freq_table)

#### Encode the training data series

In [291]:
encoder = StaticANSEncoder(freq_table)
encoded_value = encoder.encode(value_to_symbol_list)

#### Decode the training data series

In [292]:
decoder = StaticANSDecoder(freq_table)
decoded_list = decoder.decode(encoded_value,len(value_to_symbol_list))

#### Convert to the symbols to values

In [293]:
data_list_after_decoder = [q.symbol_to_value(i) for i in decoded_list[::-1]]

#### Compare the real value and decode value

In [294]:
# Real value
print(data_list)
print(len(data_list))
print()
# After the decode
print(data_list_after_decoder)
print(len(data_list_after_decoder))

[156.0, 188.0, 242.0, 157.0, 183.0, 195.0, 66.0, 42.0, 170.0, 93.0, 177.0, 69.0, 78.0, 229.0, 83.0, 80.0, 218.0, 94.0, 198.0, 240.0, 93.0, 94.0, 107.0, 237.0, 11.0, 185.0, 39.0, 237.0, 75.0, 326.0, 240.0, 124.0, 265.0, 252.0, 144.0, 207.0, 141.0, 333.0, 219.0, 252.0, 221.0, 218.0, 86.0, 235.0, 66.0, 210.0, 164.0, 222.0, 195.0, 229.0, 154.0, 74.0, 171.0, 212.0, 259.0, 247.0, 191.0, 175.0, 247.0, 123.0, 217.0, 78.0, 235.0, 180.0, 223.0, 208.0, 69.0, 240.0, 233.0, 204.0, 196.0, 220.0, 272.0, 243.0, 193.0, 140.0, 110.0, 188.0, 265.0, 243.0, 239.0, 212.0, 263.0, 214.0, 224.0, 229.0, 177.0, 136.0, 182.0, 306.0, 73.0, 178.0, 157.0, 72.0, 98.0, 168.0, 207.0, 178.0, 213.0, 234.0, 247.0, 101.0, 179.0, 198.0, 172.0, 218.0, 244.0, 186.0, 233.0, 257.0, 256.0, 246.0, 155.0, 266.0, 101.0, 183.0, 250.0, 78.0, 240.0, 168.0, 216.0, 313.0, 113.0, 217.0, 214.0, 185.0, 152.0, 213.0, 229.0, 233.0, 233.0, 237.0, 82.0, 231.0, 139.0, 327.0, 84.0, 242.0, 68.0, 238.0, 349.0, 238.0, 191.0, 286.0, 260.0, 233.0, 24

#### Evaluator

In [295]:
test_data_list = test_AriviyalN_Data_Wind_Direction["Wind Direction"].to_numpy().tolist()
test_value_to_symbol_list = [q.value_to_symbol(i) for i in test_data_list]

In [296]:
e1 = Evaluator(encoder,decoder)
result = e1.evaluate(test_value_to_symbol_list)
print(result)

{'encode_time_sec': 2.1013107999997374, 'encode_mem_mb': 59.94921875, 'decode_time_sec': 1.7798793000001751, 'decode_mem_mb': 57.90234375, 'compression_ratio': 8.265566844381853, 'bits_per_symbol': 7.742965631389349, 'mse': np.float64(319710639.52565473), 'snr_db': np.float64(6.1213706551821065)}
